## Transfer Learning from CNN LSTM


### Method 2
https://medium.com/@slaxman/perform-regression-using-transfer-learning-to-predict-house-prices-97e432a66ba5

- separate chain of numerical data vs. image data as an input

In [1]:
import imp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import sys
import tensorflow as tf
from tensorflow import keras

%matplotlib inline

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

import os
tf.random.set_seed(42)


sys.path.insert(0, '../../src')
from utils import df_to_xarray,read_xarray,inverse_scale_image, get_point_prediction

sys.path.insert(0, '../../src/preprocess')
from data_preprocess import preprocess_image_reduced,preprocess_images

/burg/glab/users/sk4973/venv/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(
2022-02-09 14:32:35.613149: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2022-02-09 14:32:35.613795: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Numeric Data Loading and Preprocessing

In [27]:
# Reading Data
dir_name="../../data/data1"
chl,mld,sss,sst,u10,fg_co2,xco2,icefrac,patm,pco2=read_xarray(dir_name)

# Creating one singular df
data_read=xr.merge([mld.MLD,mld.MLD_socat,sst.SST,sst.SST_socat,sss.SSS,sss.SSS_socat,xco2])
tmp_data=data_read.to_dataframe().reset_index()

tmp_data=tmp_data.drop(columns=['bnds','TLONG', 'TLAT', 'time_bnds'])

chl_data=chl.Chl.to_dataframe().reset_index()
chl_data_socat=chl.Chl_socat.to_dataframe().reset_index()
pco2_data=pco2.pCO2.to_dataframe().reset_index()
pco2_data_socat=pco2.pCO2_socat.to_dataframe().reset_index()

tmp_data["Chl_socat"]=chl_data_socat["Chl_socat"]
tmp_data["Chl"]=chl_data["Chl"]
tmp_data["pCO2_socat"]=pco2_data_socat["pCO2_socat"]
tmp_data["pCO2"]=pco2_data["pCO2"]

features_socat = ['time', 'xlon', 'ylat','MLD_socat', 'SST_socat', 'SSS_socat','Chl_socat', 'XCO2','pCO2_socat']
features = ['time', 'xlon', 'ylat','MLD','SST','SSS','Chl','XCO2','pCO2']

# create separate dataframe for socat
combined_socat=tmp_data.loc[:,features_socat]
combined=tmp_data.loc[:,features]

# drop rows where pco2 or pco2_socat == NA
combined_socat.dropna(subset = ["pCO2_socat"], inplace=True)
combined_socat= combined_socat[combined_socat['pCO2_socat']!=0]
combined.dropna(subset = ["pCO2"], inplace=True)
combined= combined[combined['pCO2']!=0]

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

In [31]:
time_to_id = dict()

times=np.unique(combined_socat['time'])

for t in range(len(times)):
    time=times[t]
    time_to_id[time]=t

combined_socat['time']=combined_socat['time'].map(time_to_id)
combined['time']=combined['time'].map(time_to_id)

In [32]:
X_socat=combined_socat.iloc[:,3:-1]
X=combined.iloc[:,3:-1]
y=combined.loc[:,'pCO2']
y_socat=combined_socat.loc[:,'pCO2_socat']

X_socat=num_pipeline.fit_transform(X_socat)
X=num_pipeline.fit_transform(X)

## Image Data and Model Loading

In [4]:
import tensorflow.keras.backend as kb
import tensorflow as tf
from tensorflow.keras import backend as K

def custom_rmse(y_true, y_pred):
    """
    custom_rmse(y_true, y_pred)
    calculates root square mean value with focusing only on the ocean
    """
    y_pred = y_pred[y_true != 0]
    y_true = y_true[y_true != 0]
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    
    return K.sqrt(K.mean(tf.math.squared_difference(y_pred, y_true), axis=-1))

In [5]:
sys.path.insert(0, '../../src/preprocess')
from data_preprocess import preprocess_image_reduced,preprocess_images


# Image component 
dir_name="../../data/data1"
val_dir_name="../../data/data2"

data,pco2 = preprocess_images(dir_name)
data_socat, pco2_socat = preprocess_images(dir_name, socat = True)
val_data,val_pco2 = preprocess_images(val_dir_name,"035")
val_data_socat, val_pco2_socat = preprocess_images(val_dir_name,"035", socat = True)


In [35]:
#data issue
#data_socat[combined_socat['time']]

In [6]:
INPUT_SHAPE=data_socat[0].shape

INPUT_SHAPE

(180, 360, 5)

In [7]:
base_model = tf.keras.models.load_model('../../models/base_model/base_model_new.h5', custom_objects={'custom_rmse':custom_rmse})

2022-02-09 14:34:53.037668: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-09 14:34:53.138997: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2022-02-09 14:34:53.170623: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-09 14:34:53.186414: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (g020): /proc/driver/nvidia/version does not exist
2022-02-09 14:34:53.235912: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical 

In [8]:

base_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 180, 360, 64)      8064      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 180, 360, 64)      102464    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 60, 120, 64)       0         
_________________________________________________________________
dropout (Dropout)            (None, 60, 120, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 120, 128)      204928    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 120, 128)      409728    
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 180, 360, 128)     0

In [9]:
tf.keras.backend.clear_session()

https://www.tensorflow.org/tutorials/images/transfer_learning

## Taking multiple Inputs
https://www.pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/


In [11]:
# feature extraction

base_model.trainable = False

input_numeric = keras.layers.Input(shape=(32,))

input_image = keras.layers.Input(shape=INPUT_SHAPE)

In [12]:
#image portion
avg = keras.layers.GlobalAveragePooling2D()(base_model.layers[-2].output)
dense_1= keras.layers.Dense(512, activation="elu")(avg)
dense_2 = keras.layers.Dense(64, activation="elu")(dense_1)
class_output = keras.layers.Dense(8, activation="elu")(dense_2)

image_model = keras.models.Model(inputs=base_model.input, outputs=class_output)

In [16]:
#numeric portion
y = keras.layers.BatchNormalization()(input_numeric)
y = keras.layers.Dense(1024, activation="elu")(y)
y = keras.layers.Dropout(0.2)(y)

y = keras.layers.Dense(512, activation="elu")(y)
y = keras.layers.Dropout(0.25)(y)

y = keras.layers.Dense(8, activation="elu")(y)


numeric_model = keras.models.Model(inputs=input_numeric, outputs=y)


In [17]:
combined = keras.layers.concatenate([image_model.output, numeric_model.output])

In [18]:
final = keras.layers.Dense(2, activation="relu")(combined)
final = keras.layers.Dense(1, activation="linear")(final)


final_model = keras.models.Model(inputs=[image_model.input, numeric_model.input], outputs=final)

In [19]:
final_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_input (InputLayer)       [(None, 180, 360, 5) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 180, 360, 64) 8064        conv2d_input[0][0]               
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 180, 360, 64) 102464      conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 60, 120, 64)  0           conv2d_1[0][0]                   
____________________________________________________________________________________________

In [23]:
model_path="../../models/transfer_CNN_feature.h5"


final_model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam())


early_stopings = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=1, mode='min')
checkpoint =  tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
callbacks=[early_stopings,checkpoint]

epochs = 20
batch_size = 8



final_model.fit(
    x=[X_socat,data_socat], y=y_socat,epochs=epochs, batch_size=batch_size,callbacks=callbacks)

ValueError: Data cardinality is ambiguous:
  x sizes: 245577, 421
  y sizes: 245577
Make sure all arrays contain the same number of samples.